In [1]:
from pathlib import Path

from pyspark.sql.session import SparkSession
from pyspark.sql import DataFrame

MAX_MEMORY = "8g"
spark = (
    SparkSession.builder.master("local[*]")
    .appName("TripAnaliysis")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.excutor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .config("spark.sql.caseSensitive", "true")
    .getOrCreate()
)

24/05/09 20:56:04 WARN Utils: Your hostname, limhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.109 instead (on interface en0)
24/05/09 20:56:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/09 20:56:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import os
from pathlib import Path
from pyspark.sql import DataFrame
from pyspark.sql.session import SparkSession


class FileLoadInParquet:
    def __init__(self, year: int, taxi_type: str) -> None:
        self.year = year
        self.taxi_type = taxi_type
    
    def location(self) -> list[str]:
        return str(Path(os.getcwd()).parent.joinpath(f"data/{self.taxi_type}/{self.year}"))



In [6]:
from pyspark.sql.functions import col, lit
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType

# 주어진 스키마에 따라 PySpark의 데이터 타입으로 변환
schema = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", StringType(), True),
    StructField("dropoff_datetime", StringType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("rate_code", StringType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("surcharge", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
    StructField("airport_fee", DoubleType(), True)
])


In [7]:
from pyspark.sql import Column

def read_parquet_data(year: int, spark: SparkSession) -> DataFrame:
    # 파케이 파일 경로
    data: str = FileLoadInParquet(year, "YellowTaxi").location()
    return spark.read.schema(schema).option("allowExistingColumns", True).parquet(f"file:///{data}/*")

def checking_count(data: DataFrame) -> DataFrame:
    sum_null: list[Column] = [F.sum(col(c).isNull().cast(IntegerType())).alias(c) for c in data.columns]
    null_checking: None = data.agg(*sum_null).cache()

    return null_checking

for year in range(2009, 2025):
    # data = read_parquet_data(year, spark).printSchema()
    # print(data)
    checking_count(read_parquet_data(year, spark)).toPandas().to_csv(f"{year}_null_checking.csv", index=False, index_label=False)

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)

None
root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = t

In [ ]:
from pyspark.sql.types import IntegerType

def month_classification(name: str) -> DataFrame:
    return F.split(col(name), " ", )[0].alias("datetime")
    

col_name = "Trip_Pickup_DateTime"
# Replace 'data' with your DataFrame
null_check = data.withColumn(
    "year",
    F.when(col(col_name) != "null",
           F.year(month_classification(col_name)).cast(IntegerType())).otherwise(None)
).withColumn(
    "month",
    F.when(col(col_name) != "null",
           F.month(month_classification(col_name)).cast(IntegerType())).otherwise(None)
).withColumn(
    "day",
    F.when(col(col_name) != "null",
           F.dayofmonth(month_classification(col_name)).cast(IntegerType())).otherwise(None)
).select(
    col("Trip_Pickup_DateTime"),
    col("year"),
    col("month"),
    col("day"),
).groupBy(col("month"), col("day")).agg(F.count("*").alias("count")).cache()

In [ ]:
na_delect = null_check.filter(col("month").isNotNull())
filtered = null_check.select(
    "*"
).orderBy(
    F.asc(col("month")), 
    F.asc(col("day")),
).show()

In [ ]:
spark.stop()